---
sidebar_position: 3
---

# Extracting Schema for OpenAI Functions

In the code example below, we use the [openai_function_call](https://github.com/jxnl/openai_function_call) library to extract a schema to be used on [`OpenAIChatChain`](pathname:///reference/litechain/contrib/index.html#litechain.contrib.OpenAIChatChain) from a good old python function, so you don't need to write the schema yourself.

First you need to install the library:

```bash
pip install openai-function-call
```

Then simply annotate your function using `@openai_function` and you will be able to get a `openai_schema` out of it:

In [12]:
from typing import Literal, TypedDict

from openai_function_call import openai_function


class WeatherReturn(TypedDict):
    location: str
    forecast: str
    temperature: str

# highlight-next-line
@openai_function
def get_current_weather(
    location: str, format: Literal["celsius", "fahrenheit"] = "celsius"
) -> WeatherReturn:
    """
    Gets the current weather in a given location, use this function for any questions related to the weather

    Parameters
    ----------
    location
        The city to get the weather, e.g. San Francisco. Guess the location from user messages

    format
        A string with the full content of what the given role said
    """

    return WeatherReturn(
        location=location,
        forecast="sunny",
        temperature="25 C" if format == "celsius" else "77 F",
    )


get_current_weather.openai_schema

{'name': 'get_current_weather',
 'description': '\n    Gets the current weather in a given location, use this function for any questions related to the weather\n\n    Parameters\n    ----------\n    location\n        The city to get the weather, e.g. San Francisco. Guess the location from user messages\n\n    format\n        A string with the full content of what the given role said\n    ',
 'parameters': {'properties': {'location': {'type': 'string'},
   'format': {'default': 'celsius',
    'enum': ['celsius', 'fahrenheit'],
    'type': 'string'}},
  'required': ['format', 'location'],
  'type': 'object'}}

We can then use this schema directly on our chain:

In [13]:
import json
from typing import Union

from litechain import Chain, collect_final_output
from litechain.contrib import OpenAIChatChain, OpenAIChatDelta, OpenAIChatMessage

chain: Chain[str, Union[OpenAIChatDelta, WeatherReturn]] = OpenAIChatChain[
    str, OpenAIChatDelta
](
    "WeatherChain",
    lambda user_input: [
        OpenAIChatMessage(role="user", content=user_input),
    ],
    model="gpt-3.5-turbo",
    # highlight-next-line
    functions=[get_current_weather.openai_schema],
    temperature=0,
).map(
    lambda delta: get_current_weather(**json.loads(delta.content))
    if delta.role == "function" and delta.name == "get_current_weather"
    else delta
)

await collect_final_output(
    chain(
        "I'm in my appartment in Amsterdam, thinking... should I take an umbrella for my pet chicken?"
    )
)

[{'location': 'Amsterdam', 'forecast': 'sunny', 'temperature': '25 C'}]

That's it, if you have any questions about this example, [join our discord community](https://discord.gg/48ZM5KkKgw) and we can help you out.
